In [2]:
from pathlib import Path
from ray import tune
import pandas as pd
from dataclasses import dataclass
from typing import List
import numpy as np

/home/ledoyen202/.cache/pypoetry/virtualenvs/ecir2025-0R3xmSh3-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-30 09:21:11,985	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-30 09:21:15,331	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
@dataclass
class Info:
    file: str
    number_of_samples: int
    dfs: List[pd.DataFrame] = lambda x: []
    index: pd.Index = None

In [3]:
METRIC, MODE = "eval_sari_r2_bertf1_hmean", "max"
PRED_FILES = {
    "FALC": Info("predictions.json", 20), 
    "POLITIC": Info("professions_predictions.json", 10)
}

In [4]:
def get_preds(path):
    expe_analysis = tune.ExperimentAnalysis(path.resolve(), default_metric=METRIC, default_mode=MODE)
    rg = tune.ResultGrid(expe_analysis)
    ckpt_path = Path(rg
        .get_best_result(
            metric=METRIC,
            mode=MODE,
            scope="all"
        )
        .get_best_checkpoint(
            metric=METRIC,
            mode=MODE,
        ).path
    )

    return pd.concat(
        [
            (
                pd.read_json(ckpt_path/info.file, encoding="utf-8-sig")
                    .assign(dataset=k)
                    .astype(str)
            )
            for k, info in PRED_FILES.items()
        ],
        ignore_index=True
    )
    
    

In [5]:
mbarthez_df = get_preds(Path("../../mbarthez/lora_ablation/results/retrain-best-conf/"))
mistral_df = get_preds(Path("../../mistral-7B/lora/results/retrain-best-conf/"))

In [103]:
df = pd.concat(
    [
        mbarthez_df, 
        mistral_df
    ], 
    keys=["mbarthez-lora", "mistral-lora"], 
    axis=1
).stack(level=0).reset_index()

/tmp/ipykernel_853370/926704434.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = pd.concat(


In [150]:
def select_samples(df, n):
    return np.random.choice(df.level_0.unique(), size=n, replace=False)
ids = np.concatenate(df.groupby("dataset").apply(
    lambda df: select_samples(df, PRED_FILES[df.dataset.unique()[0]].number_of_samples)
))

/tmp/ipykernel_853370/2854436177.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ids = np.concatenate(df.groupby("dataset").apply(
/tmp/ipykernel_853370/2854436177.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ids = np.concatenate(df.groupby("dataset").apply(


In [152]:
ids.shape

(30,)

In [165]:
df.to_json("all_generations.json")

In [167]:
selected_samples = df[df.level_0.isin(ids)].sample(frac=1)

In [169]:
selected_samples.to_json("./selected_samples.json")

In [1]:
selected_samples

NameError: name 'selected_samples' is not defined

In [4]:
selection = pd.read_json("selected_samples.json")

In [15]:
selection

,level_0,level_1,source,target,prediction,dataset
144,72,mbarthez-lora,"PAIX ET INDÉPENDANCE. Climat, guerres, crises ...",Paix et indépendance. Les pays européens doive...,PAIX ET INDÉPENDANCE L’Europe de la paix et de...,POLITIC
99,49,mistral-lora,Un intrus dans la maison – Maow… cela faisait ...,Panique dans la maison Jules et Bou ont bien d...,Un intrus dans la maison Jules et Bou se révei...,FALC
169,84,mistral-lora,"En me faisant confance, vous voterez pour une ...","En votant pour moi, vous votez pour une France...",J'ai confiance en vous. Je vous demande de vot...,POLITIC
56,28,mbarthez-lora,Manon soupire : « Merci d'être venu à notre re...,« Coucou les amis! » « Oh! Nous sommes si heur...,Manon soupire : « Merci d'être venu à notre re...,FALC
47,23,mistral-lora,L'homme est armé. Il aperçoit Lucas et crie un...,Les chasseurs attachent Lucas et Manon. Lucas ...,Un homme arrive. L'homme a une arme. L'homme v...,FALC
8,4,mbarthez-lora,"Dans un étang s'abreuvent des girafes, des élé...","« Regarde, Flairtout! Des girafes, des éléphan...","Dans un étang, il y a des girafes, des éléphan...",FALC
103,51,mistral-lora,"Et puis, d'un coup, tout se déroule très vite....","Soudain, Bou arrive. Bou attrape la souris. « ...",Bou attrape la souris. « Bou ! » « Bou ? » « T...,FALC
12,6,mbarthez-lora,Flairtout tire son ami de l'embarras. « Commen...,« Montrez-moi sa chambre. » Lucas et Flairtout...,« Commençons par fouiller la chambre de Manon....,FALC
24,12,mbarthez-lora,Mercredi Première rencontre du matin : un guép...,« Mercredi : Quels animaux allons-nous voir ce...,Mercredi Première rencontre : un guépard perch...,FALC
43,21,mistral-lora,Profitant d'un moment où les braconniers rient...,« Manon est sûrement dans le camion. Je dois a...,Lucas entre dans le camion. Il voit Manon. « M...,FALC
